In [637]:
import pandas as pd
from IPython.display import display, HTML
from assembly_tools import format_table, extract_raw_performance
from hydra import initialize, compose
import numpy as np

### Assembly of experimental results 1

In [638]:
# We conduced a large HP search to determine which HP configurations are optimal for a certain method and dataset. 
# You can un the same HP searches by using the benchmarking script in the causal discovery zoo folder. 
# Check the readme for details. You can also reproduce the all numbers by running the selected HP combo in the same way with the benchmark script .

In [639]:
def build_exp1(scoring="Max individual F1", null_name="Null F1"):

    with initialize(version_base=None, config_path="config/"):
        cfg = compose(config_name='extract.yaml')
    hp_stack = []
    performance_stack = []

    for name in ["reverse", "cross", "combo", ]:
        data = pd.read_csv("grid_export_exp1/" + name + ".csv", index_col=0)
        # quick fix
        res = "method.filter_mode"

        data.loc[data[res].isnull(),res] = "none"
        formatted = format_table(data,cfg, scoring= scoring, restriction=[res] if res else [])
        lines = extract_raw_performance(formatted,cfg,scoring,name=name, restriction=[res])
        hp_stack.append(formatted)
        performance_stack.append(lines)
        
    for name in ["var_", "varlingam", "dynotears", "pcmci"]:
        data = pd.read_csv("grid_export_exp1/" + name + ".csv", index_col=0)
        # quick fix
        res = None
        formatted = format_table(data,cfg, scoring= scoring, restriction=[res] if res else [])
        lines = extract_raw_performance(formatted,cfg,scoring,name=name, restriction=None)
        hp_stack.append(formatted)
        performance_stack.append(lines)
        
    bs = data[[null_name, "label_path"]].groupby("label_path").max().T
    bs.index = ["Null model"]

    # load this direct as we ran this completely seperated.
    #data = load_experimental_grid(mypath=cfg.data_path + "exp1/", method_name="cdmi").T
    # data.to_csv(cfg.data_path + "exp1/cached.csv")
    data = pd.read_csv(cfg.data_path + "exp1/cached.csv", index_col=0)
    for x in cfg.rename:
        if x in data.columns:
            print("renaming..", x)
            if cfg.rename[x] in data.columns:
                data.loc[data[cfg.rename[x]].isnull(), cfg.rename[x]] = data.loc[
                    data[cfg.rename[x]].isnull(), x
                ]
                data.drop(columns=x, inplace=True)
            else:
                data.rename(columns={x: cfg.rename[x]}, inplace=True)
    data = data[[scoring, "label_path"]].groupby("label_path").mean()
    data = data.rename(index= {"../../datasets/close_3/east.p": "close_3","../../datasets/close_5/east.p":"close_5"}).loc[cfg.ds_order].T
    data.index.name = None
    data.index = ["CDMI"]
    performance_stack.append(data)

    for name in ["cp"]:
        data = pd.read_csv("grid_export_exp1/" + name + ".csv", index_col=0)
        # quick fix
        res = "method.architecture"
        cfg.ds_order = cfg.ds_order[:-1]
        formatted = format_table(data,cfg, scoring= scoring, restriction=[res] if res else [])
        lines = extract_raw_performance(formatted,cfg,scoring,name=name, restriction=[res])
        hp_stack.append(formatted)
        performance_stack.append(lines)
        
    performance_stack.append(bs)
    out = pd.concat(performance_stack)
    
    out = out.loc[out.index[[16,1,2,0,3,4,8,6,7,5,9,10,11,12,13,14,15]]]

    out.rename(index={
    "cross_none": "CC",
    "cross_corr": "CC+C",
    "reverse_none": "RP",
    "reverse_biggest": "RP+B",
    "reverse_next": "RP+N",
    "combo_none": "RPCC",
    "combo_biggest": "RPCC+B",
    "combo_next": "RPCC+N",
    "combo_corr": "RPCC+C",
    "cp_unidirectional": "CP (Gru)",
    "cp_transformer":"CP (Transf)",
    "var_": "VAR",
    "varlingam": "Varlingam",
    "pcmci": "PCMCI"
    }, inplace=True)

    out = out.astype(float)

    def highlight_max(s, props=''):
        return np.where(s == np.nanmax(s.values), props, '')


    def highlight_second_min(s, props=''):
        return np.where(s == s.iloc[1:].min(), props, '')

    def highlight_second_max(s, props=''):
        return np.where(s == s.nlargest(2).values[1], props, '')


    c1 = 'background-color:#b8f2be'
    c2 = 'background-color:#a5cfa9'
    c3 = 'background-color:#e8af97'

    styling = out.style.apply(highlight_max, props=c1, axis=0).apply(highlight_second_max, props=c2, axis=0).format(precision=2)

    print(styling.format(precision=2).to_latex(convert_css=True).replace("0.", ".").replace("nan", "\\textdagger").replace(" .", "."))


# Styling of the table

In [640]:
build_exp1(scoring="Individual AUROC", null_name="Null AUROC")

/tmp/ipykernel_3520831/855385107.py:34: DtypeWarning: Columns (22,24,25,30,54,55,58,59,61,62,63,64,66,67,75,76,77,81,82,85,86,87,88,89,90,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(cfg.data_path + "exp1/cached.csv", index_col=0)


renaming.. resolution_value
renaming.. method_hps.max_lag
renaming.. method_hps.cp_architecture
renaming.. ds_name
renaming.. normalize
renaming.. window_data_month_value
renaming.. window_data_year_value
\begin{tabular}{lrrrrrrrrrrr}
 & close_3 & close_5 & root_cause_3 & root_cause_5 & 1_random_3 & 1_random_5 & confounder_3 & confounder_5 & random_3 & random_5 & disjoint_10 \\
Null model &.50 &.50 &.50 &.50 &.50 &.50 &.50 &.50 &.50 &.50 &.50 \\
RP &.80 &.76 &.76 &.70 &.74 &.73 &.62 &.66 &.79 &.75 &.75 \\
RP+N &.72 &.62 & {\cellcolor[HTML]{B8F2BE}}.81 & {\cellcolor[HTML]{A5CFA9}}.77 &.71 &.65 &.58 &.58 &.72 &.64 &.65 \\
RP+B &.78 &.71 &.61 &.53 &.81 &.71 &.61 &.62 &.76 &.68 &.58 \\
CC &.68 &.63 &.70 &.67 &.64 &.66 &.65 &.59 &.71 &.71 &.66 \\
CC+C &.69 &.64 &.72 &.70 &.69 &.67 &.64 &.60 &.71 &.67 &.75 \\
RPCC &.70 &.66 &.70 &.65 &.68 &.68 &.63 &.60 &.72 &.71 &.71 \\
RPCC+N &.68 &.60 &.73 &.70 &.67 &.64 &.62 &.57 &.69 &.64 &.66 \\
RPCC+B &.68 &.63 &.60 &.55 &.72 &.66 &.63 &.58 &.69 &.65 

In [641]:
build_exp1(scoring="Max individual F1", null_name="Null F1")

renaming.. resolution_value
renaming.. method_hps.max_lag
renaming.. method_hps.cp_architecture
renaming.. ds_name
renaming.. normalize
renaming.. window_data_month_value
renaming.. window_data_year_value
\begin{tabular}{lrrrrrrrrrrr}
 & close_3 & close_5 & root_cause_3 & root_cause_5 & 1_random_3 & 1_random_5 & confounder_3 & confounder_5 & random_3 & random_5 & disjoint_10 \\
Null model &.50 &.34 &.50 &.33 &.29 &.26 &.54 &.36 &.50 &.33 &.16 \\
RP &.75 &.53 &.71 &.48 &.48 &.42 &.66 &.48 &.74 &.52 &.29 \\
RP+N &.68 &.46 & {\cellcolor[HTML]{B8F2BE}}.86 & {\cellcolor[HTML]{B8F2BE}}.67 &.55 &.43 &.55 &.45 &.69 &.49 &.36 \\
RP+B &.73 &.57 &.50 &.33 &.61 &.50 &.56 &.47 &.71 &.53 &.24 \\
CC &.65 &.44 &.66 &.46 &.44 &.39 &.65 &.43 &.67 &.49 &.25 \\
CC+C &.67 &.49 &.70 &.57 &.52 &.46 &.60 &.45 &.68 &.52 & {\cellcolor[HTML]{A5CFA9}}.53 \\
RPCC &.66 &.47 &.64 &.45 &.50 &.42 &.62 &.44 &.67 &.51 &.30 \\
RPCC+N &.65 &.44 &.73 &.58 &.52 &.43 &.60 &.44 &.67 &.49 &.38 \\
RPCC+B &.65 &.48 &.55 &.36 &.5

/tmp/ipykernel_3520831/855385107.py:34: DtypeWarning: Columns (22,24,25,30,54,55,58,59,61,62,63,64,66,67,75,76,77,81,82,85,86,87,88,89,90,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(cfg.data_path + "exp1/cached.csv", index_col=0)


In [642]:
build_exp1(scoring="Max individual Acc", null_name="Null Acc")

renaming.. resolution_value
renaming.. method_hps.max_lag
renaming.. method_hps.cp_architecture
renaming.. ds_name
renaming.. normalize
renaming.. window_data_month_value
renaming.. window_data_year_value
\begin{tabular}{lrrrrrrrrrrr}
 & close_3 & close_5 & root_cause_3 & root_cause_5 & 1_random_3 & 1_random_5 & confounder_3 & confounder_5 & random_3 & random_5 & disjoint_10 \\
Null model &.67 &.80 &.67 &.80 &.83 &.85 &.63 &.78 &.67 &.80 &.91 \\
RP &.80 &.80 &.79 &.80 &.83 &.85 &.76 &.78 &.80 &.80 &.91 \\
RP+N &.78 &.81 & {\cellcolor[HTML]{B8F2BE}}.90 & {\cellcolor[HTML]{B8F2BE}}.89 &.83 &.86 &.67 &.79 &.79 &.82 &.91 \\
RP+B &.82 &.85 &.67 &.80 &.83 &.87 &.70 &.80 &.80 &.83 &.91 \\
CC &.76 &.80 &.76 &.80 &.84 &.85 &.74 &.78 &.77 &.81 &.91 \\
CC+C &.78 &.83 &.80 &.85 &.84 &.86 &.72 &.80 &.79 &.83 &.92 \\
RPCC &.77 &.81 &.75 &.80 &.85 &.85 &.75 &.79 &.77 &.81 &.91 \\
RPCC+N &.77 &.82 &.82 &.86 &.85 &.86 &.73 &.80 &.79 &.83 &.91 \\
RPCC+B &.77 &.83 &.71 &.81 &.86 &.86 &.74 &.80 &.78 &.83 

/tmp/ipykernel_3520831/855385107.py:34: DtypeWarning: Columns (22,24,25,30,54,55,58,59,61,62,63,64,66,67,75,76,77,81,82,85,86,87,88,89,90,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(cfg.data_path + "exp1/cached.csv", index_col=0)


### Unused


In [ ]:
with_ranking = pd.DataFrame(
    out.astype(float).round(2).astype(str).values
    + " ("
    + out.rank(ascending=False).values.astype(int).astype(str)
    + ")",
    columns=out.columns,
    index=out.index,
)
with_ranking.style.background_gradient(cmap="YlGn_r", axis="index", vmin=0.4)

In [455]:
table = out.rank(ascending=True).style.background_gradient(cmap="YlGn_r", axis="index", vmin=0.4)
table.data = out
table.format(precision=2)